In [1]:
cd '/home/dfried/projects/onecommon/aaai2020/experiments'

/home/dfried/projects/onecommon/aaai2020/experiments


In [2]:
import models
import data
import utils
from domain import get_domain
from corpora.reference_sentence import ReferenceSentenceCorpus

In [3]:
domain = get_domain('one_common')
unk_threshold = 10
model_ty = models.get_model_type('rnn_reference_model')

ctx_encoder_type = 'rel_attn_encoder'

# model is actually in "serialized_models/{model_fname}"
# model_fname = 'rel_tsel_ref_dial_model/default_0_best.th'

# # valid_lang_loss 2.2762        valid_select_loss 0.8293        valid_select_acc 0.6859 valid_ref_loss 0.2897
# model_fname = 'rel2_tsel_ref_dial_model/hierarchical_rel-enc-2_rel-pool-max_mask-pad_1_best.th'

# # valid_lang_loss 2.2609        valid_select_loss 0.9497        valid_select_acc 0.6291         valid_ref_loss 0.3191
# model_fname = 'rel2_tsel_ref_dial_model/hierarchical_rel-enc-2_rel-pool-max_mask-pad_feed-ctx-attend_1_best.th'

# valid_lang_loss 
# # valid_lang_loss 2.2544        valid_select_loss 0.8247        valid_select_acc 0.7062         valid_ref_loss 0.2852
# model_fname = 'rel2_tsel_ref_dial_model_no-share/hierarchical_mask-pad_feed-context-attend-separate_1_best.th'

# # valid_lang_loss 2.2448        valid_select_loss 0.9233        valid_select_acc 0.6455         valid_ref_loss 0.3180
# model_fname = 'rel2_tsel_ref_dial_model_no-share/hierarchical_mask-pad_selection-attention_feed-context-attend-separate_no-word-attention_1_best.th'

# model_fname = 'rel2_tsel_ref_dial_model_no-share/hierarchical_mask-pad_selection-attention_feed-context-attend-separate_1_best.th'

# valid_lang_loss 2.2435        valid_select_loss 1.0267        valid_select_acc 0.6358         valid_ref_loss 0.3528
# model_fname = 'rel2_tsel_ref_dial_model_separate/hierarchical_mask-pad_max-pool_rel-2_feed-context-attend_no-word-attention_ablate-properties_1_best.th'

# model_fname = 'rel2_dial_model_separate/hierarchical_mask-pad_max-pool_rel-2_feed-context-attend_1_best.th'
# model_fname = 'rel2_dial_model_separate/hierarchical_mask-pad_max-pool_rel-2_feed-context-attend_no-word-attention_1_best.th'
# model_fname = 'rel2_dial_model_separate/hierarchical_mask-pad_max-pool_rel-2_feed-context-attend_no-word-attention_ablate-properties_1_best.th'
# model_fname = 'rel2_dial_model_separate/hierarchical_mask-pad_max-pool_feed-context-attend_no-word-attention_ablate-properties_1_best.th'
# model_fname = 'rel2_dial_model_separate/hierarchical_mask-pad_max-pool_feed-context-attend_ablate-properties_1_best.th'

# model_fname = 'rel2_dial_model_separate/hierarchical_mask-pad_max-pool_feed-context-attend_ablate-properties_1_best.th'

model_fname = 'rel2_dial_model_separate/hierarchical_only-self_mask-pad_max-pool_feed-context-attend_ablate-properties_1_best.th'
# model_fname = 'rel2_dial_model_separate/hierarchical_only-self_mask-pad_max-pool_rel-2_feed-context-attend_no-word-attention_ablate-properties_1_best.th'

In [4]:
# has diverse outputs, but doesn't attend to meaningful things
# model_fname = 'rel2_dial_model_separate/hierarchical_only-self_constrain-attn_mask-pad_max-pool_ablate-properties_1_best.th'

# feed attention is peaked, word attention doesn't attend to meaningful things
# model_fname = 'rel2_dial_model_separate/hierarchical_only-self_constrain-attn_mask-pad_feed-context-attend_1_best.th'

# model_fname = 'rel2_dial_model_separate/hierarchical_only-self_constrain-attn_mask-pad_max-pool_feed-context-attend_no-word-attention_ablate-properties_1_best.th'
# model_fname = 'rel2_dial_model_separate/hierarchical_only-self_constrain-attn_mask-pad_max-pool_feed-context-attend_no-word-attention_ablate-properties_1_best.th'

# model_fname = 'rel2_dial_model_separate/hierarchical_only-self_constrain-attn_mask-pad_max-pool_rel-2_feed-context-attend_no-word-attention_1_best.th'
model_fname = 'rel2_dial_model_separate/hierarchical_only-self_constrain-attn_mask-pad_max-pool_rel-2_feed-context-attend_1_best.th'

In [5]:
model_fname = 'rel2_dial_model_separate/hierarchical_only-self_supervise-feed-attn_mask-pad_feed-context-attend_1_best.th'

In [6]:
seed = 1
corpus = ReferenceSentenceCorpus(domain, 'data/onecommon',
                            train='train_reference_{}.txt'.format(seed),
                            valid='valid_reference_{}.txt'.format(seed),
                            test='test_reference_{}.txt'.format(seed),
                            freq_cutoff=unk_threshold, verbose=True)

dataset data/onecommon/train_reference_1.txt, total 545056, unks 4302, ratio 0.79%
dataset data/onecommon/valid_reference_1.txt, total 69348, unks 686, ratio 0.99%
dataset data/onecommon/test_reference_1.txt, total 67284, unks 636, ratio 0.95%


In [7]:
def load_json(file):
    import json
    with open(file, 'r') as f:
        return json.load(f)

In [8]:
scenarios1 = load_json('data/onecommon/aaai_train_scenarios.json')
scenarios2 = load_json('data/onecommon/aaai_train_scenarios_2.json')

In [9]:
len(scenarios1)

9000

In [10]:
len(scenarios2)

4500

In [11]:
scenarios1[0].keys()

dict_keys(['kbs', 'shared', 'uuid'])

In [12]:
train_scenarios = {
    scenario['uuid']: scenario
    for scenario in scenarios1
}

In [13]:
hit_count = 0
miss_count = 0
for scenario in scenarios2:
    uuid = scenario['uuid']
    if uuid in train_scenarios:
        assert train_scenarios[uuid] == scenario
        hit_count += 1
    else:
        miss_count += 1

In [14]:
print("hits: {}".format(hit_count))
print("misses: {}".format(miss_count))

hits: 0
misses: 4500


In [15]:
corpus

In [16]:
model = utils.load_model(model_fname, map_location='cpu')

In [17]:
batchsize=1
validset, validset_stats = corpus.valid_dataset(batchsize)

In [18]:
validset_stats

{'n': 74030, 'nonpadn': 69348}

In [19]:
import torch

In [20]:
from agent import RnnAgent

/home/dfried/anaconda2/envs/onecommon/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [21]:
from selfplay import make_parser

In [22]:
parser = make_parser()
args = parser.parse_args('--temperature 0.25'.split())
# args = parser.parse_args('--temperature 0.01'.split())

In [23]:
alice = RnnAgent(model, args)
bob = RnnAgent(model, args)

In [24]:
def is_selection(out):
    return '<selection>' in out

In [25]:
# display

In [26]:
from dialog import DialogLogger

In [27]:
from IPython.display import SVG, display, HTML

In [28]:
def display_svgs(svgs):
    no_wrap_div = '<div style="white-space: nowrap">' + ''.join(svgs) + '</div>'
    display(HTML(no_wrap_div))

In [29]:
def display_attn(scenario, attn, agent_id, name=None):
    attn = attn.flatten().detach().numpy()
    if name is not None:
        print("{}: {}".format(name, attn))
    display_svgs([DialogLogger._scenario_to_svg(scenario)[agent_id], DialogLogger._attention_to_svg(scenario, agent_id, attn)])

In [30]:
pdb on

Automatic pdb calling has been turned ON


In [31]:
# training emulation

In [32]:
trainset, trainset_stats = corpus.train_dataset(bsz=1)

In [33]:
trainset[0].scenario_ids[0] in train_scenarios

True

In [34]:
# ctx: this player's dots
# inpt / tgt: dialogue word indices (tgt are shifted by 1)
# ref_inpt

In [35]:
for ix in range(len(trainset)):
    ctx, inpts, tgts, ref_inpts, ref_tgts, sel_tgt, \
    [scenario_id], real_ids, agents, chat_ids, sel_idxs, \
    lens, rev_idxs, hid_idxs, all_num_markables = trainset[ix]
#     if 'C_d0de3d0144df405e998ae718539ff9a4' in chat_ids:
    if scenario_id == 'S_UoujG1GG6KscmnbN':
        print("found it")
        scenario = train_scenarios[scenario_id]
        break

found it


KeyError: 'S_UoujG1GG6KscmnbN'

> <ipython-input-35-da5e076431c6>(8)<module>()
      5 #     if 'C_d0de3d0144df405e998ae718539ff9a4' in chat_ids:
      6     if scenario_id == 'S_UoujG1GG6KscmnbN':
      7         print("found it")
----> 8         scenario = train_scenarios[scenario_id]
      9         break

ipdb> q


In [48]:
real_ids

[['5', '24', '35', '39', '40', '58', '80']]

In [49]:
ref_tgts[1]

In [50]:
scenario

{'kbs': [[{'color': 'rgb(75,75,75)', 'x': 124, 'id': '5', 'size': 9, 'y': 239},
   {'color': 'rgb(97,97,97)', 'x': 133, 'id': '24', 'size': 11, 'y': 189},
   {'color': 'rgb(177,177,177)', 'x': 352, 'id': '33', 'size': 7, 'y': 276},
   {'color': 'rgb(149,149,149)', 'x': 202, 'id': '35', 'size': 8, 'y': 169},
   {'color': 'rgb(139,139,139)', 'x': 81, 'id': '39', 'size': 7, 'y': 130},
   {'color': 'rgb(76,76,76)', 'x': 231, 'id': '40', 'size': 10, 'y': 112},
   {'color': 'rgb(85,85,85)', 'x': 228, 'id': '80', 'size': 9, 'y': 319}],
  [{'color': 'rgb(75,75,75)', 'x': 199, 'id': '5', 'size': 9, 'y': 227},
   {'color': 'rgb(97,97,97)', 'x': 208, 'id': '24', 'size': 11, 'y': 177},
   {'color': 'rgb(149,149,149)', 'x': 277, 'id': '35', 'size': 8, 'y': 157},
   {'color': 'rgb(139,139,139)', 'x': 156, 'id': '39', 'size': 7, 'y': 118},
   {'color': 'rgb(76,76,76)', 'x': 306, 'id': '40', 'size': 10, 'y': 100},
   {'color': 'rgb(115,115,115)', 'x': 83, 'id': '58', 'size': 9, 'y': 255},
   {'color':

In [51]:
inpts[0].size()

torch.Size([16, 1])

In [52]:
corpus.word_dict

In [53]:
real_ids

[['5', '24', '35', '39', '40', '58', '80']]

In [54]:
scenario['kbs'][0]

[{'color': 'rgb(75,75,75)', 'x': 124, 'id': '5', 'size': 9, 'y': 239},
 {'color': 'rgb(97,97,97)', 'x': 133, 'id': '24', 'size': 11, 'y': 189},
 {'color': 'rgb(177,177,177)', 'x': 352, 'id': '33', 'size': 7, 'y': 276},
 {'color': 'rgb(149,149,149)', 'x': 202, 'id': '35', 'size': 8, 'y': 169},
 {'color': 'rgb(139,139,139)', 'x': 81, 'id': '39', 'size': 7, 'y': 130},
 {'color': 'rgb(76,76,76)', 'x': 231, 'id': '40', 'size': 10, 'y': 112},
 {'color': 'rgb(85,85,85)', 'x': 228, 'id': '80', 'size': 9, 'y': 319}]

In [55]:
agents[0]

1

In [56]:
ref_tgts[0]

tensor([[[0, 1, 0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 0],
         [0, 1, 0, 0, 0, 0, 0]]])

In [57]:
# training emulation
scenario = train_scenarios[scenario_id]
agent = alice
agent.feed_context(ctx.flatten())
agent.real_ids = real_ids
agent.agent_id = agents[0]
    
samples = 10
    
writer = alice
conv = []
speaker = []
words_left = 5000

sentence_ix = 0

assert len(inpts) == len(tgts)
    
while sentence_ix < len(inpts):
    words_og = [corpus.word_dict.idx2word[ix] for ix in inpts[sentence_ix].flatten().numpy()]
#     for _ in range(samples):
#         pred_outs, _, _, _, extra = writer.model.write(writer.ctx_h, writer.lang_h, words_left, writer.args.temperature)
#         print('{}\t{}'.format(writer.agent_id, ' '.join(writer._decode(outs.narrow(0,1,outs.size(0)-1), writer.model.word_dict))))
    print(' '.join(words_og))
    if ref_inpts[sentence_ix] is not None:
        dots_mentioned = (ref_tgts[sentence_ix].sum(dim=1) > 0)
        display_attn(scenario, dots_mentioned.float(), writer.agent_id, name='dots_mentioned')
    else:
        dots_mentioned = None
    out = writer.write(
        max_words=words_left,
        force_words=[words_og],
        start_token=words_og[0],
        dots_mentioned=dots_mentioned
    )
#     out = writer.write(max_words=words_left)
    
    if writer.extras[-1]['dot_attention1'] is not None:
        display_attn(scenario, writer.extras[-1]['dot_attention1'], writer.agent_id, name='feed attention')
    
    if 'dot_attention2_mean' in writer.extras[-1]:
        display_attn(scenario, writer.extras[-1]['dot_attention2_mean'], writer.agent_id, name='word attention mean')
        
    for ix, ix_attn in enumerate(writer.extras[-1]['dot_attention2s']):
        display_attn(scenario, ix_attn, writer.agent_id, name='t {}: {}'.format(ix, out[ix]))
        
    print('{}\t{}'.format(writer.agent_id, ' '.join(out)))
    print()
    words_left -= len(out)
    conv.append(out)
    speaker.append(writer.agent_id)
#     reader.read(out)
    if is_selection(out) or words_left <= 1:
        break
#     writer, reader = reader, writer
    sentence_ix += 1

YOU: i see a dark grey dot with a slightly smaller darker dot below it <eos>
dots_mentioned: [1. 1. 0. 0. 0. 0. 0.]


feed attention: [0.21415079 0.26355082 0.20366506 0.10438601 0.11483226 0.03861998
 0.06079509]


word attention mean: [0.21117762 0.18205652 0.09723911 0.09005012 0.18244098 0.10613222
 0.13090344]


t 0: i: [0.20555297 0.21779485 0.0682409  0.06809045 0.18552554 0.11448714
 0.14030816]


t 1: see: [0.15178424 0.15108328 0.12941994 0.13272911 0.15402313 0.1356761
 0.14528424]


...

t 0: ok: [0.15567748 0.1738715  0.10888808 0.10632848 0.15353547 0.15242183
 0.1492771 ]


t 1: good: [0.17266373 0.17710336 0.08647171 0.07979087 0.16702901 0.1542011
 0.16274025]


t 2: luck: [0.18017115 0.17240942 0.09705767 0.10843942 0.16469511 0.13576041
 0.14146678]


t 3: <selection>: [0.17580864 0.17606011 0.08287815 0.08132374 0.1668407  0.16274422
 0.1543444 ]


1	ok good luck <selection>



In [97]:
import json
from utils import ContextGenerator

In [98]:
ctx_gen = ContextGenerator('data/onecommon/shared_4.txt')

In [99]:
with open('data/onecommon/shared_4.json', "r") as f:
    scenario_list = json.load(f)
selfplay_scenarios = {scenario['uuid']: scenario for scenario in scenario_list}

In [100]:
scenario = None
for ix, ([scenario_id], (ctx1, ctx2), (ids1, ids2)) in enumerate(ctx_gen.iter()):
#     if scenario_id == 'S_11J8n40u4HRVduwi':
#         print('found it')
#         scenario = scenarios[scenario_id]
#         break
    if ix == 0:
        scenario = selfplay_scenarios[scenario_id]
        print(scenario_id)
        break

S_grq8fYvbxiPZnvlM


In [101]:
# selfplay emulation
scenario = selfplay_scenarios[scenario_id]
for agent, agent_id, ctx, real_ids in zip([alice, bob], [0, 1], [ctx1, ctx2], [ids1, ids2]):
    agent.feed_context(ctx)
    agent.real_ids = real_ids
    agent.agent_id = agent_id
    
samples = 10
    
writer, reader = [alice, bob]
conv = []
speaker = []
words_left = 5000
    
while True:
    for _ in range(samples):
        outs, _, _, _, extra = writer.model.write(writer.ctx_h, writer.lang_h, words_left, writer.args.temperature)
        print('{}\t{}'.format(writer.agent_id, ' '.join(writer._decode(outs.narrow(0,1,outs.size(0)-1), writer.model.word_dict))))
    out = writer.write(max_words=words_left)
    print('{}\t{}'.format(writer.agent_id, ' '.join(out)))
    print()
    
    if writer.extras[-1].get('dot_attention1', None) is not None:
        display_attn(scenario, writer.extras[-1]['dot_attention1'], writer.agent_id, name='feed attention')
    
    if writer.extras[-1].get('dot_attention2_mean', None) is not None:
        display_attn(scenario, writer.extras[-1]['dot_attention2_mean'], writer.agent_id, name='word attention mean')
        
#     for ix, ix_attn in enumerate(writer.extras[-1]['dot_attention2s']):
#         display_attn(scenario, ix_attn, writer.agent_id, name='t {}: {}'.format(ix, out[ix]))
    print('{}\t{}'.format(writer.agent_id, ' '.join(out)))
    print()
        
    words_left -= len(out)
    conv.append(out)
    speaker.append(writer.agent_id)
    reader.read(out)
    if is_selection(out) or words_left <= 1:
        break
    writer, reader = reader, writer

0	i have a tiny black dot with a larger lighter dot to the left of it <eos>
0	i have a large light grey dot with a smaller lighter dot to the right of it <eos>
0	i have a large light grey dot with a slightly smaller and lighter dot to the right and slightly below it <eos>
0	i have a large light gray dot with a smaller dark dot below and to the left of it <eos>
0	i have a large grey dot with a smaller lighter dot to the left and below it <eos>
0	i have a large light grey dot with a smaller lighter dot below it and to the left of it <eos>
0	i have a large light grey dot with a slightly smaller one slightly to the right and slightly to the left <eos>
0	i have a large light grey dot with a smaller dark dot above it <eos>
0	i have a large light grey dot with a slightly smaller one above it and to the right <eos>
0	i have a large grey dot with a smaller dark dot to the right and slightly to the right <eos>
0	do you have a large light grey dot with a smaller lighter grey dot to the left and s

0	do you have a large light grey dot with a smaller lighter grey dot to the left and slightly above it <eos>

1	i have a large light grey dot with a smaller black dot to the right and slightly below it <eos>
1	yes i see that ! let 's pick the large one <selection>
1	yes i see that . let 's pick the larger one <selection>
1	i have a tiny black dot with a large light grey dot to the right of it <eos>
1	yes i see that . let 's choose the large one <selection>
1	i have a large light grey dot with a smaller black dot to the left of it <eos>
1	i do n't have that . i have a large light grey dot with a smaller black dot to the left and slightly above it <eos>
1	i see a tiny black dot with a large light grey dot up and to the right of it <eos>
1	i see that . i have a large light grey dot with a smaller black dot below and to the right of it <eos>
1	yes , i see that too . let 's pick the large one <selection>
1	yes , i see that . <eos>

word attention mean: [0.03539151 0.08812761 0.07640824 0.10

1	yes , i see that . <eos>

0	ok lets choose the lighter one <selection>
0	lets pick the larger one <selection>
0	ok , lets pick the large one <selection>
0	ok let 's pick the larger one <selection>
0	ok lets pick the large light one <selection>
0	ok lets pick the larger one <selection>
0	let 's pick the large one <selection>
0	ok lets pick the larger one <selection>
0	ok lets pick the large light grey dot <selection>
0	ok lets pick the larger one <selection>
0	ok lets click the larger one <selection>

word attention mean: [0.4412072  0.01331483 0.02236627 0.01403854 0.24668472 0.11929139
 0.14309704]


0	ok lets click the larger one <selection>

